In [1]:
# Import necessary libraries
from keras.models import load_model
from picamera import PiCamera
import cv2
import os
import tkinter
from tkinter import messagebox
import smtplib
import time # Provides time-related functions
import cv2 # OpenCV library
import numpy as np

from imutils import paths

In [2]:
# Initialize Tkinter
root = tkinter.Tk()
root.withdraw()

''

In [10]:
# Create Object for Camera
camera = PiCamera()

NameError: name 'PiCamera' is not defined

In [3]:
# Load trained deep learning model
model = load_model('face_mask_detection_system.h5')

In [4]:
# Classifier to detect face
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
# While Loop to continuosly detect camera feed

# Capture Video
vid_source = cv2.VideoCapture(0)

In [6]:
# Dictionaries containing details of wearing Mask and color of rectangle around face
# Green and if not wearing mask then color of rectangle around face would be red.

text_dict = {0:'Mask ON',1:'No Mask'}
rect_color_dict = {0:(0,255,0),1:(0,0,255)}

In [7]:
SUBJECT = "Subject"
TEXT = "One Visitor violated Face Mask Policy. See in the camera to recognize user A person has been detected without a face mask in the Hotel Lobby area. Please Alert the authorities.Photo Captured:"

In [11]:
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
data = []
while True:
    flag, img = capture.read()
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if flag:
        faces = face_det_classifier.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1, -1)
            face_img = grayscale_img[y:y+w,x:x+w]
            resized_img = cv2.resize(face_img,(112,112))
            normalized_img = resized_img/255.0
            reshaped_img = np.reshape(normalized_img,(1,112,112,1))
            result = model.predict(reshaped_img)

            label = np.argmax(result, axis = 1)[0]

            cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
            cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
            cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)

        cv2.imshow('LIVE Video Feed', img)
        #27 - ASCII of Escape
        if cv2.waitKey(2) == 27:
            break

capture.release()
cv2.destroyAllWindows()

In [ ]:
# While Loop to continuously detect camera feed
while (True):
    ret, img = vid_source.read()
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img, 1.3,5)
    
    for (x,y,w,h) in faces:
        
        face_img = grayscale_img[y:y+w, x:x+w]
        resized_img = cv2.resize(face_img,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result = model.predict(reshaped_img)
     
        label = np.argmax(result, axis = 1)[0]

        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
            
        # If label = 1 then it means wearing No Mask and 0 means wearing Mask
        if (label == 1):
            # Throw a Warning Message to tell user to wear mask if not wearing one.
            # Open and No Access will be given He/She wears the mask
            messagebox.showwarning("Warning","Access Denied. Please wear a Face Mask")

            # Send an Email to the Administrator if access denied/user not wearing face mask
            message = 'Subject:{}\n\n{}'.format(SUBJECT,TEXT)
            mail = smtplib.SMTP('smtp.gmail.com',587)
            mail.ehlo()
            mail.starttls()
            mail.login("sagweombui25@gmail.com","Ombuigeoffrey23#")
            mail.sendmail("sagweombui25@gmail.com","sagweombui25@gmail.com", message)
            mail.close
        else:
            pass
            break

        # Clear the stream in preparation for the next frame
        cv2.imshow('LIVE Video Feed', img)
        #27 - ASCII of Escape
        if cv2.waitKey(2) == 27:
            break

capture.release()
cv2.destroyAllWindows()